## 제8장 CAE (Convolutional Auto-Encoder)를 이용한 노이즈 제거

**공통 모듈 불러오기**

In [1]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda

**예제 코드8.1:MNIST 데이터 불러오기와 전처리**

In [2]:
from tensorflow.python.keras.datasets import mnist

(x_train, _), (x_test, _) = mnist.load_data() 
# CNN에서 사용하기 위한 형태로 변환
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
# 이미지를 0~1의 범위로 정규화
x_train = x_train/255. 
x_test = x_test/255. 

**예제 코드8.2:마스킹 노이즈를 추가해서 인위적인 노이즈 데이터 생성**

In [3]:
def make_masking_noise_data(data_x, percent=0.1):
    size = data_x.shape
    masking = np.random.binomial(n=1, p=percent, size=size)
    return data_x*masking

x_train_masked = make_masking_noise_data(x_train)
x_test_masked = make_masking_noise_data(x_test)

**예제 코드8.3:가우시안 노이즈를 추가해서 인위적인 노이즈 데이터 생성**

In [4]:
def make_gaussian_noise_data(data_x, scale=0.8):
    gaussian_data_x = data_x + np.random.normal(loc=0, scale=scale, size=data_x.shape)
    gaussian_data_x = np.clip(gaussian_data_x, 0, 1)
    return gaussian_data_x

x_train_gauss = make_gaussian_noise_data(x_train)
x_test_gauss = make_gaussian_noise_data(x_test)

**예제 코드8.4:노이즈를 추가한 이미지와 원래 이미지 비교**

In [5]:
from IPython.display import display_png

display_png(array_to_img(x_train[0]))
display_png(array_to_img(x_train_gauss[0]))
display_png(array_to_img(x_train_masked[0]))

**예제 코드8.5:CAE 모델 구축**

In [6]:
autoencoder = Sequential()

# Encoder 부분
autoencoder.add(Conv2D(16, (3, 3), 1, activation='relu', padding='same', input_shape=(28, 28, 1)))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))

autoencoder.add(Conv2D(8, (3, 3), 1, activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))

# Decoder부분
autoencoder.add(Conv2D(8, (3, 3), 1, activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))

autoencoder.add(Conv2D(16, (3, 3), 1, activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))

autoencoder.add(Conv2D(1, (3, 3), 1, activation='sigmoid', padding='same'))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

initial_weights = autoencoder.get_weights()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


**예제 코드8.6 모델 요약 확인**

In [7]:
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 16)        1

**예제 코드8.7:가우시안 노이즈 데이터를 이용한 학습**

In [8]:
autoencoder.fit(
                x_train_gauss, # 입력: 가우시안 노이즈 데이터
                x_train,       # 정답 : 원래 이미지
                epochs=10,     # 학습할 에폭 수
                batch_size=20, # 배치 사이즈
                shuffle=True   # 데이터를 셔플한다
               )

Epoch 1/10
60000/60000 [==============================] - 13s 210us/sample - loss: 0.1810
Epoch 2/10
60000/60000 [==============================] - 11s 190us/sample - loss: 0.1574
Epoch 3/10
60000/60000 [==============================] - 12s 199us/sample - loss: 0.1531
Epoch 4/10
60000/60000 [==============================] - 12s 205us/sample - loss: 0.1505
Epoch 5/10
60000/60000 [==============================] - 13s 209us/sample - loss: 0.1487
Epoch 6/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.1476
Epoch 7/10
60000/60000 [==============================] - 12s 197us/sample - loss: 0.1468
Epoch 8/10
60000/60000 [==============================] - 11s 181us/sample - loss: 0.1462
Epoch 9/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.1458
Epoch 10/10
60000/60000 [==============================] - 11s 179us/sample - loss: 0.1454


**예제 코드8.8:가우시안 노이즈 데이터로 학습한 모델의 예측**

In [9]:
gauss_preds = autoencoder.predict(x_test_gauss)

**예제 코드8.9:가우시안 노이즈 이미지, CAE 모델, 원래 이미지의 예측 비교**

In [10]:
for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_gauss[i]))
    display_png(array_to_img(gauss_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


**예제 코드8.10:CAE모델의 가중치를 초기화**

In [11]:
autoencoder.set_weights(initial_weights)

**예제 코드8.11:마스킹 노이즈 데이터를 이용해서 학습**

In [12]:
autoencoder.fit(
                x_train_masked, # 입력 : 마스킹 노이즈 데이터
                x_train,        # 정답 : 원래 이미지
                epochs=10,      # 학습할 에폭 횟수
                batch_size=20,  # 배치 사이즈
                shuffle=True    # 데이터를 셔플한다
               )

Epoch 1/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.2073
Epoch 2/10
60000/60000 [==============================] - 11s 183us/sample - loss: 0.1911
Epoch 3/10
60000/60000 [==============================] - 11s 184us/sample - loss: 0.1876
Epoch 4/10
60000/60000 [==============================] - 12s 195us/sample - loss: 0.1855
Epoch 5/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.1844
Epoch 6/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.1836
Epoch 7/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.1829
Epoch 8/10
60000/60000 [==============================] - 12s 204us/sample - loss: 0.1825
Epoch 9/10
60000/60000 [==============================] - 12s 199us/sample - loss: 0.1820
Epoch 10/10
60000/60000 [==============================] - 11s 187us/sample - loss: 0.1817


**예제 코드8.12:마스킹 노이즈 데이터로 학습한 모델의 예측**

In [13]:
masked_preds = autoencoder.predict(x_test_masked)

**예제 코드8.13:마스킹 노이즈 이미지, CAE 모델, 원래 이미지의 예측 비교**

In [14]:
for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_masked[i]))
    display_png(array_to_img(masked_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------
